# The Timeseries Quantum Autoencoder
*TS QAE in Qiskit with angle encoding of TS sliding windows*

By: Jacob Cybulski<br>
Date: August 2023 - February 2024<br>
Aims: The goal of this notebook is to build a Time Series Quantum Autoencoder, 
    a circuit which can compress a quantum state of a timeseries onto a smaller amount of qubits, 
    while retaining the information from the initial state.
Methods: A QAE model from Qiskit Tutorial 12 has been adopted for time series processing. 
- Time series data was converted to a series of sliding windows.
- Several approaches to TS data encoding were tested, i.e. unary, binary and anglular.
- Angle encoding was eventually used, with values centered around H state, in the range of [-pi/2..pi/2]
- The model featuring an input and encoder blocks only (followed by a swap test) was subsequently trained.
- A decoder was then added for testing, and the circuit initialised with an optimum set of parameters from training.
- Each test sample was then encoded into the full QAE circuit, which was executed using a state vector simulation.
- State vectors of input and output data was then visualised and compared. 

Sources: 
1. Romero, Jonathan, Jonathan P. Olson, and Alan Aspuru-Guzik. 2017. “Quantum Autoencoders for Efficient Compression of Quantum Data.” Quantum Science and Technology 2 (4): 045001.
2. Bravo-Prieto, Carlos, "Quantum autoencoders with enhanced data encoding." Machine Learning: Science and Technology, 2, May 2021
3. Qiskit Tutorial, https://qiskit.org/ecosystem/machine-learning/tutorials/12_quantum_autoencoder.html. *Based on [1].*
4. Eugenia Anello, Denoising Autoencoder in Pytorch on MNIST dataset, Dataseries (Medium), Jun 28, 2021.
5. Eugenia Anello, <a href="https://github.com/eugeniaring/Medium-Articles/blob/main/Pytorch/denAE.ipynb">GitHub</a>, Jun 28, 2021.
6. Phillip Lippe, Tutorial 9: Deep Autoencoders, UvA Deep Learning Tutorials, Fall 2022.

Notes:
- Refer to the end of the notebook for the log of changes
- Algorithm by Romera was followed
- Results of the enhanced algorithm by Bravo-Prieto could not be reproduced

## Initial settings

In [ ]:
%%html
<style>
table {float:left}
</style>

In [ ]:
import sys
sys.path.append('.')
sys.path.append('..')
sys.path

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pylab
import math
from IPython.display import clear_output
%matplotlib inline

from utils.Target import *
from utils.Window import *
from utils.Callback import *
from utils.TS import *

## Prepare windowed TS data
*Values need to be in [-1..+1] range*

### Prepare time series

In [ ]:
### Start random process
from qiskit_algorithms.utils import algorithm_globals
algorithm_globals.random_seed = 2022

In [ ]:
### Data settings
samples = 80
samples_hi_res = samples*2
split = 0.75

### Define the time series parameters
#   wind_size: sliding window size (which may be the same as number of qubits)
#   horizon:   the number of time events to look ahead (to forecast)
wind_size = 8 # tested with: 6(-), 8(+), 9(-) 10(++)
wind_step = 4
horizon = 1
scale = 1 # 2
pan = 0 # -1

### Noise level
noise_org = 0.03
noise_ts = 0.03


### Utility TS plot

In [ ]:
### Plot source data
def qae_plot_source_data(
    X_train, y_train, X_valid, y_valid,
    add_markers=True,
    label_suffix=['', '', ''],
    xlabel='Range', ylabel='Target value (deltas)',
    title=f'Differenced TS Windows for Training and Validation',
    sel_wind=None):

    # Plot the original time series
    plt.rcParams["figure.figsize"] = (12, 6)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    # plt.xlim(lb, ub)

    # Plot target function
    plt.plot([xt[0] for xt in X_train], [y[0] for y in y_train], color='blue', label='Training')
    plt.plot([xv[0] for xv in X_valid], [y[0] for y in y_valid], color='red', label='Validation')
    if sel_wind != None:
        plt.plot(X_train[sel_wind], y_train[sel_wind], color='magenta', label='Window '+str(sel_wind))
    if add_markers:
        plt.plot([xt[0] for xt in X_train], [y[0] for y in y_train], marker='o', color='lightblue', linestyle='None')
        plt.plot([xv[0] for xv in X_valid], [y[0] for y in y_valid], marker='o', color='pink', linestyle='None')
    plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.2),
              ncol=3, fancybox=True, shadow=True)
    plt.show()


### Generate TS with options

In [ ]:
### Select a target class and its parameters (see: utils.py), e.g.
#   Target_sin() # Target_2_sins() # Target_poly() # Target_poly_3()
#   Target_line() # Target_line(slope=0.5, intercept=0.2, xmin=0, xmax=1.5)
#   Target_trig_trend() # Target_jitter()
#   Target_beer() # Target_beer(pt_from=104, pt_to=156)

fun_train = Target_beer(pt_from=104, pt_to=156) # Target_2_sins() # Target_poly_3()
sel_wind = 3

X_train_org, y_train_org, X_valid_org, y_valid_org = \
    gen_ts_windows(fun_train, samples_hi_res, split, wind_size, wind_step, differencing=False)

X_train_org_noisy, y_train_org_noisy, X_valid_org_noisy, y_valid_org_noisy = \
    gen_ts_windows(fun_train, samples_hi_res, split, wind_size, wind_step, differencing=False, noise=noise_org)

# fun_train = Target_2_sins()
X_train_ts, y_train_ts, X_valid_ts, y_valid_ts = \
    gen_ts_windows(fun_train, samples_hi_res, split, wind_size, wind_step, differencing=True)

X_train_noisy, y_train_noisy, X_valid_noisy, y_valid_noisy = \
    gen_ts_windows(fun_train, samples_hi_res, split, wind_size, wind_step, differencing=True, noise=noise_ts)

### Plot all TSs
qae_plot_source_data(X_train_org, y_train_org, X_valid_org, y_valid_org, sel_wind=sel_wind, add_markers=False,
                    title='Original time series', ylabel='Target value')
qae_plot_source_data(X_train_org_noisy, y_train_org_noisy, X_valid_org_noisy, y_valid_org_noisy, sel_wind=sel_wind, add_markers=False,
                    title=f'Original time series with noise ({noise_org})', ylabel='Target value')
# qae_plot_source_data(X_train_ts, y_train_ts, X_valid_ts, y_valid_ts, sel_wind=100, add_markers=False,
#                     title=f'Differenced pure time series')
# qae_plot_source_data(X_train_noisy, y_train_noisy, X_valid_noisy, y_valid_noisy, sel_wind=100, add_markers=False,
#                     title=f'Differenced time series with noise ({noise_ts})')


### Utilities

In [ ]:
from IPython.display import clear_output
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, plot_bloch_vector, plot_state_city, plot_state_paulivec
from qiskit import Aer

In [ ]:
# Converts measurements amplitudes a and b in a|0>+b|1> into bloch vector
def convert_amps_to_xyz(a, b):
    x = 2*np.real(a*np.conjugate(b))
    y = 2*np.imag(b*np.conjugate(a))
    z = a*np.conjugate(a)-b*np.conjugate(b)
    return [x, y, z] 

### Angle encoding
Deltas between consecutive time series values have been angle encoded. In the context of a quibit representation (see the Figure), the encoding assumes zero to be encoded as H state, negative values to be rotations up, while positive values as rotations down. This encoding allows cumulative sequence calculations and easy value decoding upong the qubit measurements. Should there be huge voilatility in data, additional scaling has been added to shrink the region of valid angular qubit positions. 

<div>
    <!--img src="attachment:qae_fig2_wide.png" width="1000"-->
    <img src="../images/seq-value-encoding.png" width="300">
</div>

**Figure: Sequence value coding as qubit angular rotations**

In [ ]:
### Angle encoding of a TS value relative to the previous value
#   val: value in [-1..+1] range to be encoded
#   optional scaler=np.pi/2: Number scaler 
#   optional err_range=0/0.05: allows range scaling to cater for accumulating errors
#   returns: Encoding of the next value relative to the previous value
def ts_relang_encode_val(val, scaler=np.pi/2, err_range=0):
    return val * scaler * (1 - 2 * err_range)

### Decoding
def ts_relang_decode_val(val, scaler=np.pi/2, err_range=0):
    return val / (scaler * (1 - 2 * err_range))

### Normalises the value to 1 (not required with angle encoding)
def ts_relang_norm_val(next_code):
    norm_code = next_code
    return norm_code

### Print encoding and decoding for testing
def print_ts_relang_encode_val(n):
    val = round(ts_relang_encode_val(n), 3)
    if val == 0:
        print(f'{(n)} -> {val} (π*{0.0})')
    else:
        print(f'{(n)} -> {val} (π/{round(np.pi / val, 3)})')
    
def print_ts_relang_decode_val(n):
    if n == 0:
        print(f'{round(n, 3)} (π*{0.0}) -> {ts_relang_decode_val(n)}')
    else:
        print(f'{round(n, 3)} (π/{round(np.pi / n, 3)}) -> {round(ts_relang_decode_val(n), 3)}')

def print_ts_relang_norm_val(p):
    print(f'{round(p, 3)} -> {round(ts_relang_norm_val(p), 3)}')

### Test
# print('Encoding:')
# print_ts_relang_encode_val(0)
# print_ts_relang_encode_val(0.5)
# print_ts_relang_encode_val(0.7)
# print_ts_relang_encode_val(0.25)
# print_ts_relang_encode_val(1.2) # Out of [0..pi] range
# print_ts_relang_encode_val(-0.7) # Out of [0..pi] range

# print('\nDecoding:')
# print_ts_relang_decode_val(np.pi)
# print_ts_relang_decode_val(0*np.pi)
# print_ts_relang_decode_val(np.pi/4)
# print_ts_relang_decode_val(-np.pi/2)
# print_ts_relang_decode_val(np.pi/8)
# print_ts_relang_decode_val(-0.9*np.pi) # Out of [0..pi] range

# print('\nNormalise Encoding:')
# print_ts_relang_norm_val(1.5*np.pi)
# print_ts_relang_norm_val(-0.5*np.pi)

In [ ]:
### Encoding the entire data set
def ts_relang_encode(wind_set, scaler=np.pi):
    encoded_set = []
    for wind_idx in range(wind_set.shape[0]):
        wind = wind_set[wind_idx]
        encoded_wind = []
        for val_idx in range(wind.shape[0]):
            val = wind[val_idx]
            encoded_val = ts_relang_encode_val(val, scaler)
            encoded_wind.append(encoded_val)
        encoded_set.append(encoded_wind)
    org_wind_start = np.array([w[0] for w in wind_set])
    return np.array(encoded_set), org_wind_start

### Encoding the entire data set
def ts_relang_decode(org_wind_start, encoded_wind_set, scaler=np.pi):
    decoded_set = []
    for wind_idx in range(encoded_wind_set.shape[0]):
        wind = encoded_wind_set[wind_idx]
        decoded_wind = []
        for val_idx in range(wind.shape[0]):
            encoded_val = wind[val_idx]
            decoded_val = ts_relang_decode_val(encoded_val, scaler)
            decoded_wind.append(decoded_val)
            prev = decoded_val
        decoded_set.append(decoded_wind)
    return np.array(decoded_set)

### In agle encoding, encoded data is already normalised
def ts_relang_norm(encoded_wind_set):
    return np.array(encoded_wind_set)

### Preparing data for training and testing

In [ ]:
### Encoding of TS windows which are 
y_train_enc, org_train_start = ts_relang_encode(y_train_ts)
y_valid_enc, org_valid_start = ts_relang_encode(y_valid_ts)

y_train_noisy_enc, org_train_noisy_start = ts_relang_encode(y_train_noisy) 
y_valid_noisy_enc, org_valid_noisy_start = ts_relang_encode(y_valid_noisy)

### Testing validation windows

# y_train_dec = ts_relang_decode(org_train_start, ts_relang_norm(y_train_enc))
# print('\nTraining windows before encoding:\n', y_train_ts[5:8])
# print('\nTraining windows after encoding:\n', y_train_enc[5:8])
# print('\nTraining windows org start:\n', org_train_start[5:8])
# print('\nTraining windows after decoding:\n', y_train_dec[5:8])

# y_valid_dec = ts_relang_decode(org_valid_start, y_valid_enc)
# print('\nValidation windows before encoding:\n', y_valid_ts[0:8])
# print('\nValidation windows after encoding:\n', y_valid_enc[0:8])
# print('\nValidation windows org start:\n', org_valid_start[0:8])
# print('\nValidation windows after decoding:\n', y_valid_dec[0:8])


In [ ]:
# Plot deltas in the time series
plt.rcParams["figure.figsize"] = (12, 6)
plt.title(f"Differenced Training Data (noise={noise_ts} added to original data)")
plt.xlabel("Range (only 1st sliding window elements)")
plt.ylabel("Target value (deltas)")

sel_range=range(len(y_train_enc))
#sel_range=range(70, 100, 1)
#sel_range=range(10, 40, 1)

plt.plot(list(sel_range), [y[0] for y in y_train_enc[sel_range]], color='blue', label='Training data (pure)')
plt.plot(list(sel_range), [y[0] for y in y_train_noisy_enc[sel_range]], color='red', label='Training data (with noise)')
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.2),
          ncol=2, fancybox=True, shadow=True)
plt.show()

In [ ]:
# Plot deltas in the time series
plt.rcParams["figure.figsize"] = (12, 6)
plt.title(f"Differenced Testing Data (noise={noise_ts} added to original data)")
plt.xlabel("Range (only 1st sliding window elements)")
plt.ylabel("Target value (deltas)")

sel_range=range(len(y_valid_enc))
#sel_range=range(70, 100, 1)
#sel_range=range(10, 40, 1)

plt.plot(list(sel_range), [y[0] for y in y_valid_enc[sel_range]], color='blue', label='Testing data (pure)')
plt.plot(list(sel_range), [y[0] for y in y_valid_noisy_enc[sel_range]], color='red', label='Testing data (with noise)')
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.2),
          ncol=2, fancybox=True, shadow=True)
plt.show()

## Building and training a Quantum Autoencoder

In [ ]:
### Libraries used in QAE development

import json
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit
from qiskit_algorithms.optimizers import COBYLA, NELDER_MEAD, SPSA
from qiskit.circuit.library import RealAmplitudes, TwoLocal, ZFeatureMap, ZZFeatureMap
from qiskit.quantum_info import Statevector
from qiskit_algorithms.utils import algorithm_globals
from qiskit.visualization import plot_histogram, plot_state_city, plot_state_paulivec
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit.circuit import Parameter

### Sequence encoder
The sequence is encoded as a series of angle rotations

In [ ]:
### Creates a circuit encoding a sequence 
#   - Adds H gates to position each qubit at a "zero" position
#   - Each sequence value y-rotates the qubit state either up (negative) or down (positive)
#   - Use only as many input values as the number of qubits, if not enough provided zero rotations are added
#
#   qubits_no: Number of qubits
#   input_no: Number of values in a sequence
#   optional classreg=False: Flag indicating if a classical register is to be added
#   optional meas_q=None: Indicates the qubit to be measured, active only when classreg=True
#   err_added: Indicates percent of error to be generated
#   returns: Circuit encoding a sequence
def sequence_encoder(qubit_no, input_no=None, classreg=False, meas_q=None, label='S'):

    if input_no == None: 
        input_no = qubit_no
    used_sgates = 0

    qr = QuantumRegister(qubit_no, 'q')
    cr = ClassicalRegister(1, 'meas')
    if classreg:
        seq = QuantumCircuit(qr, cr, name='sequence')
    else:
        seq = QuantumCircuit(qr, name='sequence')

    # Data-encoding circuit block, packs different input vars
    for q in range(qubit_no):
        seq.h(q)
        if q > input_no:
            seq.ry(0, q)
        else:
            param_x = Parameter(label+'('+str(used_sgates)+')') if used_sgates < input_no else 0
            seq.ry(param_x, q)
        used_sgates += 1

    if classreg and meas_q != None:
        seq.measure(meas_q, 0)

    return seq, seq.parameters[:]

### Test a sample sequence encoder
sample_seq, inp_params = sequence_encoder(7, 6, classreg=True, meas_q=2, label='In')
print('\n')
display(sample_seq.draw(output='latex', style="iqp", fold=False, scale=0.7))
print('\nInput parameters for the sequence: ', inp_params)

**Figure: Example of encoding for a sample sequence window**

### Creation of an ansatz

The selectede ansatz is the Qiskit RealAmplitude ansatz (2-local circuit) - the prepared quantum states will only have real amplitudes, and does not rely on full connectivity between qubits. 

In [ ]:
### Creates an ansatz to be used for QAE Encoder/Decoder
#   num_latent: size of the latent area
#   num_trash: size of the trash area
#   reps: number of repeating layers
#   ent: type of entanglement layer (linear, reverse_linear, full, circular, sca, pairwise)

# Standard ansatz based Ry and Cx
def ansatz(num_qubits, reps=3, ent='sca', insert_barriers=False, label='A'):
    anz = RealAmplitudes(num_qubits, reps=reps, entanglement=ent, insert_barriers=insert_barriers,
                        parameter_prefix=label)
    return anz

### Test the selected ansatz
sample_ansatz = ansatz(7, reps=2, ent='sca', insert_barriers=True) # linear, reverse_linear, full, circular, sca
print('\n')
display(sample_ansatz.decompose().draw("latex", style="iqp", scale=0.7, fold=False))
print(f'\nNumber of ansatz parameters: {len(sample_ansatz.parameters)}\n')

**Figure: Sample QAE ansatz**

### Swap Test
For two qubts, it returns a squared inner product between their states. In this implementation, it estimates the overlap between the states of all participating qubits. 
The repeated measurement of 1 indicates that the quantum states are identical. For two qubits, if the measurement returns 0.5 then the two states are orthogonal.
However, orthogonality of the remaining states is no longer possible, so the measurements will be further away from 1, possibly confusing the outcome.

In [ ]:
### Swap test circuit
def swap_test(num_trash):
    qr = QuantumRegister(2 * num_trash + 1, "swap")
    cr = ClassicalRegister(1, "c")
    swap_qc = QuantumCircuit(qr, cr)
    auxiliary_qubit = 2 * num_trash
    
    swap_qc.h(auxiliary_qubit)
    for i in range(num_trash):
        swap_qc.cswap(auxiliary_qubit, i, num_trash + i)    
    swap_qc.h(auxiliary_qubit)
    swap_qc.measure(auxiliary_qubit, cr[0])
    return swap_qc

### Test a swap test circuit
sample_swap_qc = swap_test(2)
print(f'\nExample swap test circuit: {len(sample_swap_qc.qubits)}\n')
display(sample_swap_qc.draw("latex", style="iqp", scale=1))

### Training Half-QAE Encoder: Input + Encoder + Swap Test
It determines the states of all trash qubits to be $\lvert 0 \rangle$ via a swap test, which returns 1 when they are all identical, and equal distribution of expectation values, when they are highly dissimilar. This is the most reliable assessment of state similarity, however, it is slow and duplicates the number of qubits in a circuit.

In [ ]:
def train_encoder_with_swap_test(num_latent, num_trash, reps=4, ent='circular',
                  seq_name='Input', seq_label='N', 
                  anz_name='Encoder', anz_label='X'):
    qr = QuantumRegister(num_latent + 2 * num_trash + 1, "q")
    cr = ClassicalRegister(1, "c")
    fm_qc, _ = sequence_encoder(num_latent+num_trash, label=seq_label)
    fm_qc.name = seq_name
    anz_qc = ansatz(num_latent+num_trash, reps=reps, ent=ent, label=anz_label).decompose()
    anz_qc.name = anz_name
    swap_qc = swap_test(num_trash)
    swap_qc.name = 'Swap'
    
    qc = QuantumCircuit(qr, cr)
    qc.append(fm_qc, qargs=range(num_latent+num_trash))
    qc.barrier()
    qc.append(anz_qc, qargs=range(num_latent+num_trash))
    qc.barrier()
    qc.append(swap_qc, qargs=range(num_latent, num_latent + 2*num_trash+1), cargs=[0])
    qc.barrier()

    return qc, fm_qc.parameters, anz_qc.parameters

### Test an encoder circuit
sample_lat=5; sample_trash=2
sample_aec, sample_fm_parameters, sample_anz_parameters = \
    train_encoder_with_swap_test(sample_lat, sample_trash, reps=2, ent='sca', seq_label='S', seq_name='Sequence')

# print(f'\nExample autoencoder circuit (with {sample_lat} latent and {sample_trash} trash qubits):')
# print(f'\nEncoder parameters:\n{sample_anz_parameters}')
# print(f'\nIn/Out  parameters:\n{sample_fm_parameters}\n\n')

display(sample_aec.decompose().draw("latex", style="iqp", scale=1, cregbundle=False))

**Figure: Half-QAE structure for training using only Encoder and the Swap Test**<br/>
The swap test compares the states of qubits in the trash space with qubits initialised with $\lvert 0 \rangle$

### Training Half-QAE Encoder: Input + Encoder without a Swap Test
It assesses the state of all trash qubits to be $\lvert 0 \rangle$ by their direct measurement and estimating the probability $P(\lvert 0 \rangle)^n$ (where $n$ is the number of qubits in the trash space. This approach to measuring qubit similarity is not as nuanced as what's is provided by swap test, as we miss on the state proximity determined by their inner product. However, the measurement is fast(er) and does not need additional qubits. The circuit training needs the cost function $cost = P(1-\lvert 0 \rangle)^n$, which needs to be minimised.

In [ ]:
### Building partial QAE for its training without a swap test

def train_encoder(num_latent, num_trash, reps=4, ent='circular',
                  seq_name='Input', seq_label='N', 
                  anz_name='Encoder', anz_label='X'):
    qr = QuantumRegister(num_latent + num_trash, "q")
    cr = ClassicalRegister(num_trash, "c")
    fm_qc, _ = sequence_encoder(num_latent+num_trash, label=seq_label)
    fm_qc.name = seq_name
    anz_qc = ansatz(num_latent+num_trash, reps=reps, ent=ent, label=anz_label).decompose()
    anz_qc.name = anz_name

    
    circuit = QuantumCircuit(qr, cr)

    ### Sequence
    circuit.append(fm_qc, qargs=range(num_latent+num_trash))

    ### Encoder
    circuit.barrier()
    circuit.append(anz_qc, qargs=range(num_latent + num_trash))
    
    ### Measurements
    circuit.barrier()
    for i in range(num_trash):
        circuit.measure(qr[num_latent+i], cr[i])
    
    return circuit, fm_qc, anz_qc

### Test a training circuit
sample_ae, _, _ = train_encoder(5, 2, reps=2, ent='sca')
print('\nExample autoencoder circuit:\n')
display(sample_ae.draw("latex", style="iqp", scale=1))

**Figure: Half-QAE structure for training using only Encoder and measurements of qubit states**<br/>
The $\lvert 0 \rangle$ state of trash qubits is determined by the cost function: $cost = 1-P(\lvert 0 \rangle^n)$ (with n - number of qubits in trash space)

### Training Full-Circuit: Input + Encoder + Decoder + Output + Swap Test
*Note: Position of the output block may vary<br/>Note: Decoder could be an inverse of encoder or become an independent block*

In [ ]:
def train_qae(num_latent, num_trash, reps=4, ent='sca',
              in_seq_name='Input', in_seq_label='I', 
              out_seq_name='Output', out_seq_label='O', 
              enc_name='Encoder', enc_label='X',
              dec_name='Decoder', dec_label='Y',
              keep_encoder=False):

    ### Create a circuit and its components
    lt_qubits = num_latent+num_trash
    qr = QuantumRegister(lt_qubits, "q")
    cr = ClassicalRegister(lt_qubits, "c")
    in_qc, _ = sequence_encoder(lt_qubits, label=in_seq_label)
    in_qc.name = in_seq_name
    out_qc, _ = sequence_encoder(lt_qubits, label=out_seq_label)
    out_qc.name = out_seq_name
    enc_qc = ansatz(lt_qubits, reps=reps, ent=ent, label=enc_label).decompose()
    enc_qc.name = enc_name
    dec_qc = ansatz(lt_qubits, reps=reps, ent=ent, label=dec_label).decompose()
    dec_qc.name = dec_name

    ### Input
    qc = QuantumCircuit(qr, cr)
    qc.append(in_qc, qargs=range(lt_qubits))

    ### Encoder
    qc.barrier()
    qc.append(enc_qc, qargs=range(lt_qubits))

    ### Latent / Trash
    qc.barrier()
    for i in range(num_trash):
        qc.reset(num_latent + i)

    ### Decoder
    qc.barrier()
    if keep_encoder:
        dec_inv_qc = enc_qc.inverse()
    else:
        dec_inv_qc = dec_qc.inverse()
    qc.append(dec_inv_qc, qargs=range(lt_qubits))

    ### Inverted output (trans input)
    qc.barrier()
    out_inv_qc = out_qc.inverse()
    qc.append(out_inv_qc, qargs=range(lt_qubits))

    ### Measurements
    qc.barrier()
    for i in range(len(qc.qubits)):
        qc.measure(qr[i], cr[i])
    
    ### Collect weight parameters
    train_weight_params = []
    for enc_p in enc_qc.parameters:
        train_weight_params.append(enc_p)    
    if not keep_encoder:
        for dec_p in dec_qc.parameters:
            train_weight_params.append(dec_p)

    ### Collect in/out parameters
    in_out_params = []
    for in_p in in_qc.parameters:
        in_out_params.append(in_p)    
    for out_p in out_qc.parameters:
        in_out_params.append(out_p)        
    
    if keep_encoder:
        return qc, in_out_params, enc_qc.parameters, enc_qc.parameters, train_weight_params
    else:
        return qc, in_out_params, enc_qc.parameters, dec_qc.parameters, train_weight_params

### Test an encoder circuit
sample_tqae, in_out_params, enc_params, dec_params, all_weight_params = \
    train_qae(5, 2, reps=2, ent='sca', in_seq_label='N', in_seq_name='Noisy Input', keep_encoder=False)
print('\nExample autoencoder circuit'+
      '\nNote: Decoder is either an inverse of the encoder or an independent decoder block):')

# print(f'\nIn/Out  parameters:\n{in_out_params}')
# print(f'\nEncoder parameters:\n{enc_params}')
# print(f'\nDecoder parameters:\n{dec_params}')
# print(f'\nWeight parameters:\n{all_weight_params}\n\n')

# display(sample_tqae.decompose().draw("latex", style="iqp", scale=0.8, cregbundle=False))
display(sample_tqae.draw("latex", style="iqp", scale=0.8, fold=-1)) # latex, mpl, 
print()

**Figure: Full-QAE structure for training (using Encoder+Decoder and a Swap Test)**<br/>
*The swap test is performed on all input/output qubits to determine the value of our cost function.<br/><br/>
Note: Position of the output block may vary<br/>Note: Decoder could be an inverse of encoder or become an independent block*

### Testing Circuit - The entire QAE

The full Autoencoder consists of both the Encoder and Decoder, which is simply an inverted Encoder. 
Both the Encoder and Decoder can be initialised using the same parameters obtained from the Encoder (plus swap test) training.
By applying the full QAE circuit to a test dataset, we can then determine the model accuracy.

In [ ]:
### Full QAE for testing the previously trained model

def qae(lat_no, trash_no, reps=2, ent='sca', classreg=False, meas_q=None, keep_encoder=False,
          in_seq_name='Input', in_seq_label='I', 
          enc_name='Encoder', enc_label='X',
          dec_name='Decoder', dec_label='Y'):

    # Prepare a circuit
    qr = QuantumRegister(lat_no + trash_no, 'q')
    cr = ClassicalRegister(1, 'meas')
    # qae_qc = QuantumCircuit(lat_no + trash_no, 1)
    if classreg:
        qae_qc = QuantumCircuit(qr, cr, name='qae')
    else:
        qae_qc = QuantumCircuit(qr, name='qae')

    # Create all QAE components
    in_qc, _ = sequence_encoder(lat_no + trash_no, label=in_seq_label)
    in_qc.name = in_seq_name
    enc_qc = ansatz(lat_no+trash_no, reps=reps, ent=ent, label=enc_label)
    enc_qc.name = enc_name
    dec_qc = ansatz(lat_no+trash_no, reps=reps, ent=ent, label=dec_label)
    dec_qc.name = dec_name

    # Create a circuit
    qae_qc.append(in_qc, qargs=range(lat_no + trash_no))
    qae_qc.barrier()
    qae_qc.append(enc_qc, qargs=range(lat_no + trash_no))
    qae_qc.barrier()
    
    for i in range(trash_no):
        qae_qc.reset(lat_no + i)
    
    qae_qc.barrier()
    if keep_encoder:
        dec_inv_qc = enc_qc.inverse()
    else:
        dec_inv_qc = dec_qc.inverse()
    qae_qc.append(dec_inv_qc, qargs=range(lat_no + trash_no))

    # Add optional measurement
    if classreg and meas_q != None:
        qae_qc.barrier()
        qae_qc.measure(meas_q, 0)

    return qae_qc, in_qc, enc_qc, dec_qc

sample_qae_qc, sample_in_qc, sample_enc_qc, sample_dec_qc = qae(5, 2, reps=2, ent='sca', classreg=True, meas_q=2, keep_encoder=False)

### Show a sample test circuit - the sequence feature map and the ansatz will beused further
print()
print('\n#QAE Input Parameters:', len(sample_in_qc.parameters))
print('#Encoder/Decoder Weights:', len(sample_enc_qc.parameters)+len(sample_dec_qc.parameters))
print('#QAE Total Weights:', len(sample_qae_qc.parameters))
print('\n')

display(sample_qae_qc.draw("latex", style="iqp", scale=1))


**Figure: Full QAE capable of mapping input sequence to its output (after Encoder/Decoder training)**

*Note: Decoder could be an inverse of encoder or become an independent block*

## Model training

**Sampler** needs to be imported from *qiskit_aer.primitives*.
Unfortunately, *qiskit-aer-gpu 0.12.2* has a bug, which has been fixed 
in the next version *qiskit-aer-gpu 0.13.0*, see notes in the version table.

### Simulator device

In [ ]:
### Find what devices are available
from qiskit_aer.backends import AerSimulator
devices = AerSimulator().available_devices()
devices

In [ ]:
## Force the CPU
# devices = ('CPU')
# devices

### Sampler options

In [ ]:
### Create a sampler suitable for the device
#   GPU methods: statevector, tensor_network, density_matrix, unitary

from qiskit.primitives import Sampler # For "CPU", ignores device="GPU" option
from qiskit_aer.primitives import Sampler as AerSampler # For device="GPU" option

seed = 2023
algorithm_globals.random_seed = 42

# Use GPU when present, otherwise CPU
if 'GPU' in devices:
    device = 'GPU'
    sampler = AerSampler(
        backend_options={'seed_simulator': seed, 'method': 'statevector', 
                         'device' : device, 'cuStateVec_enable' : True},
        run_options={'seed': seed, 'shots': 1000},
        transpile_options={"seed_transpiler": seed},
    )
else:
    device = 'CPU'
    sampler = Sampler()
    sampler.set_options(method='statevector')
    sampler.set_options(device=device)
    sampler.set_options(seed=seed)
    sampler.options

print(f'Selected device: {device}')

### Performance metrics

In [ ]:
### Various metrics calculated between two TS window sets given as dictionaries

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

### Merge a window set given as a dictionary into a list of its entries
def merged_tswind(wind_dict, trim_left=0, trim_right=0):    
    wind_list = []
    sorted_keys = sorted(wind_dict.keys())
    if (trim_left+trim_right) <= len(wind_dict[sorted_keys[0]]):
        for sel_wind in sorted_keys:
            wind = wind_dict[sel_wind]
            wind = wind[trim_left:]
            wind = wind[:-trim_right] if trim_right>0 else wind
            wind_list.extend(wind)
    return wind_list

### Perform RMS on dictionaries
def rms_tswin(wind_exp, wind_pred, trim_left=0, trim_right=0):
    exp = merged_tswind(wind_exp, trim_left=trim_left, trim_right=trim_right)
    pred = merged_tswind(wind_pred, trim_left=trim_left, trim_right=trim_right)
    return np.sqrt(mean_squared_error(exp, pred))    

### Perform MAE on dictionaries
def mae_tswin(wind_exp, wind_pred, trim_left=0, trim_right=0):
    exp = merged_tswind(wind_exp, trim_left=trim_left, trim_right=trim_right)
    pred = merged_tswind(wind_pred, trim_left=trim_left, trim_right=trim_right)
    return mean_absolute_error(exp, pred)    

### Perform MAPE on dictionaries
def mape_tswin(wind_exp, wind_pred, trim_left=0, trim_right=0):
    exp = merged_tswind(wind_exp, trim_left=trim_left, trim_right=trim_right)
    pred = merged_tswind(wind_pred, trim_left=trim_left, trim_right=trim_right)
    return mean_absolute_percentage_error(exp, pred)    

### Calculate R score on dictionaries
def r2_tswin(wind_exp, wind_pred, trim_left=0, trim_right=0):
    exp = merged_tswind(wind_exp, trim_left=trim_left, trim_right=trim_right)
    pred = merged_tswind(wind_pred, trim_left=trim_left, trim_right=trim_right)
    return r2_score(exp, pred)    

### Sample data
tsw_exp = {}
tsw_exp[2] = [2, 5, 9, 20, 21]
tsw_exp[5] = [5, 8, 9, 10, 11]
tsw_exp[1] = [1, 2, 3, 4, 5]
tsw_exp[3] = [3, 9, 11, 12, 13]

tsw_pred = {}
tsw_pred[2] = [3, 6, 10, 21, 20]
tsw_pred[5] = [6, 9, 10, 11, 12]
tsw_pred[1] = [2, 3, 4, 5, 4]
tsw_pred[3] = [4, 10, 12, 13, 14]

### Testing merge_tswind
# print('\nTesting merged_tswind')
# print(f'\tOriginal dict: {tsw_exp}')
# print(f'\tMerged dict: {merged_tswind(tsw_exp)}')
# print(f'\tMerged trim 2 left: {merged_tswind(tsw_exp, trim_left=2)}')
# print(f'\tMerged trim 1 right: {merged_tswind(tsw_exp, trim_right=1)}')
# print(f'\tMerged trim 2 left and 1 right: {merged_tswind(tsw_exp, trim_left=2, trim_right=1)}')
# print(f'\tMerged trim 3 left and 3 right: {merged_tswind(tsw_exp, trim_left=3, trim_right=3)}')
# print()

### Testing metrics
# print('\nTesting metrics')
# print(f'\tRMS: {rms_tswin(tsw_exp, tsw_pred)}')
# print(f'\tMAE: {mae_tswin(tsw_exp, tsw_pred)}')
# print(f'\tMAPE: {mape_tswin(tsw_exp, tsw_pred)}')
# print(f'\tR2: {r2_tswin(tsw_exp, tsw_pred)}')

### Cost function
We build our cost function, based on the swap test between the reference and trash space for the digit dataset. To do this, we again use Qiskit's CircuitQNN network and use the same interpret function as we are measuring the probability of getting the final qubit in the $|1\rangle$ state.

In [ ]:
from sklearn.utils import shuffle as data_shuffle
default_backend = Aer.get_backend('aer_simulator_statevector')

### Converts numbers to their binary representation
def digit_list(a, n_bits):
    return np.array([int(i) for i in f'{a:0{n_bits}b}'])

### Calculates an inner join on binary representation
def inner_join(a, b, n_bits):
    inna = digit_list(a, n_bits)
    innb = digit_list(b, n_bits)
    return np.inner(inna, innb)

### Makes an array indexed by number and 
#   returns the number of binary ones in its representation
def make_1s_count(bin_digs):
    dig_arr = [0]*bin_digs
    for i in range(bin_digs):
        dig_arr[i] = sum(digit_list(i, bin_digs))
    return np.array(dig_arr)

### Counts the probability of all measurements identical to zero
def cost_swap(probs):
    recs = probs.shape[0]
    return np.sum(probs[:, 1]) / recs

### Counts the probability of measurements which is not all zeros
def cost_zero(probs):
    recs = probs.shape[0]
    return 1.0 - np.sum(probs[:, 0]) / recs

### Counts the probability weighted number of 1s per record
def cost_min1s(probs):
    digit_no = int(np.log2(probs.shape[1]+1))
    recs = probs.shape[0]
    digits_wsum = 0
    for rec in range(recs):
        for i in range(digit_no):
            digits_wsum += sum(digit_list(i, digit_no))*probs[rec, i]
    return digits_wsum

# make the plot nicer
plt.rcParams["figure.figsize"] = (12, 6) 

### Class Cost
class Cost:
    name = "Cost class"
    
    # Initialises the costs
    def __init__(self, train_set, 
                 optimizer, init_vals, shuffle=False, shuffle_interv=10, 
                 cost_type='min1s', # cost_type = min1s, swap, zeroes
                 yscale='linear', # yscale = linear, log / asinh, function, functionlog, logit, symlog
                ):
        self.shuffle = shuffle
        self.shuffle_interv = shuffle_interv
        self.objective_func_vals = []
        self.opt = optimizer
        self.init_vals = init_vals
        self.iter = 0
        self.rand = 2023
        self.init_time = time.time()
        self.elapsed_time = 0
        self.cost_type = cost_type
        self.yscale = yscale
        self.perform_tests=False

        if self.shuffle:
            self.train_set = data_shuffle(train_set, random_state=self.rand)
        else:
            self.train_set = train_set

    ### Set testing
    def init_testing(self, valid_set, train_noisy_set, valid_noisy_set,
                     num_latent=8, num_trash=2, reps=2, ent='sca', backend=default_backend, shots=10000
                    ):
        self.perform_tests=True
        self.valid_set = valid_set
        self.params = []
        self.mae_train_vals = []
        self.mae_valid_vals = []
        self.num_latent=num_latent
        self.num_trash=num_trash
        self.reps=reps
        self.ent=ent
        self.backend=backend,
        self.shots=shots
    return
        
    ### Reset objective function values
    def reset_objfuncs(self):
        self.objective_func_vals = []
        self.mae_train_vals = []
        self.mae_valid_vals = []
        self.params = []
        self.elapsed_time = 0
        self.init_time = time.time()

    ### Plot cost
    def cost_plot(self):
        min_cost = min(self.objective_func_vals)
        clear_output(wait=True)
        plt.title(f'Cost function value against iteration (iter# {self.iter}, '+
                  f'min cost={np.round(min_cost, 4)}, '+
                  f'elapsed time={np.round(self.elapsed_time, 0)} secs)')
        plt.xlabel("Iteration")
        plt.ylabel("Cost function value")
        plt.yscale(self.yscale)
        plt.plot(range(len(self.objective_func_vals)), self.objective_func_vals)
        plt.show()

    ### Plot MAE
    def cost_mae_plot(self):
        # clear_output(wait=True)
        plt.title(f'R2 against iteration (iter# {self.iter}, '+
                  f'max R2={np.round(max_r2, 4)}, '+
                  f'elapsed time={np.round(self.elapsed_time, 0)} secs)')
        plt.xlabel("Iteration")
        plt.ylabel("MAE")
        plt.yscale(self.yscale)
        plt.plot(range(len(self.r2_train_vals)), self.r2_vals)
        plt.show()

    ### Circuit test
    def test(self, sel_wind_set, winds_pure, winds_noise):
        in_org_set = {}
        out_reconstr_set = {}
    
        for wind_idx in range(len(sel_wind_set)):
        
            sel_wind = sel_wind_set[wind_idx]    
            wind = winds_noise[sel_wind] # Use noisy window for manipulation            

            ### Find the QAE resulting window state vector representation
            out_meas = []
            for meas_q in range(len(wind)):
                last_params = self.params(len(self.params)-1)
                param_values = np.concatenate((wind, last_params))       
                out_qc, _, _, _ = qae(self.num_latent, self.num_trash, 
                                      reps=self.reps, ent=self.ent, 
                                      classreg=True, keep_encoder=False, meas_q=meas_q)     
                out_qc = out_qc.assign_parameters(param_values)
                out_qc = out_qc.decompose().decompose()
                meas_angle = single_qubit_angle_meas(out_qc, self.backend, shots=self.shots)
                out_meas.append(meas_angle)
        
            ### Add window results
            in_org_set[sel_wind] = list(winds_pure[sel_wind]) # Add pure window for reference
            out_reconstr_set[sel_wind] = out_meas

        ### Calculate the MAE for the selected windows
        mae_inorg_outrec = mae_tswin(in_org_set, out_reconstr_set, trim_left=0, trim_right=0)
        
    return mae_inorg_outrec

    ### Cost function used in training
    def cost_fun(self, params_values, *args):
        self.iter = self.iter+1
        
        if (self.shuffle) and (self.iter % self.shuffle_interv == 0):
            # print(f'*** Shuffled at iteration {self.iter}')
            self.train_set = data_shuffle(self.train_set, random_state=self.rand)
            
        probs = qnn.forward(self.train_set, params_values)
        if self.cost_type=='swap':
            cost = np.sum(probs[:, 1]) / self.train_set.shape[0] # Checking for P(|0>) on inner product
        elif self.cost_type == 'min1s':
            cost = cost_min1s(probs)
        else: # it is 'zeros'
            cost = 1.0 - np.sum(probs[:, 0]) / self.train_set.shape[0] # Checking for P(|0>^n)
        self.objective_func_vals.append(cost)
        self.params.append(params_values)

        ### Test the current model performance
        train_mae = self.test(self.sel_train_wind_set, self.train_set, self.train_noisy_set)
        self.mae_train_vals.append(train_mae)
        valid_mae = self.test(self.sel_valid_wind_set, self.valid_set, self.valid_noisy_set)
        self.mae_valid_vals.append(valid_mae)

        self.elapsed_time = time.time() - self.init_time
    
        # plotting part
        self.cost_plot()
    
        return cost

    def optimize(self):
        return opt.minimize(fun=self.cost_fun, x0=self.init_vals)


### Measuring the state of a single qubit in a circuit (in terms of its y angle)
Recall that the value returned is in relation to the H state, which is zero,<br/>
with rotations to the left interpretaed as negative values, and rotatations to the right as positive values.

In [ ]:
### Runs a circuit with a single qubit measured and returns its angular position
#   Recall the figure explaining encoding

def single_qubit_angle_meas(qc, backend, shots=10000):
    job = backend.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts(qc)
    
    counts0 = counts['0'] if '0' in counts.keys() else 0
    counts1 = counts['1'] if '1' in counts.keys() else 0
    p0 = counts0/(counts0+counts1)
    p1 = counts1/(counts0+counts1)
    amp0 = np.sqrt(p0)
    amp1 = np.sqrt(p1)

    meas_angle = 2*np.arccos(amp0)-np.pi/2
    return meas_angle

# Test
# qc = QuantumCircuit(2, 1)
# qc.ry(np.pi/2, 0)
# qc.ry(np.pi/4, 1)
# qc.measure(1, 0)
# print()
# display(qc.draw("latex", style="iqp", scale=1))
# angle = single_qubit_angle_meas(qc, backend)
# round(angle / np.pi, 2)

### Constants guiding QAE training

In [ ]:
reps=2
ent='sca' # linear, reverse_linear, full, circular, sca, pairwise
num_qubits = wind_size
num_trash = 3
num_latent = num_qubits-num_trash
cost_type = 'zeroes' # min1s, swap, zeroes
cost_yscale = 'linear' # linear, log
epochs = 500 
noise = noise_ts # Noise ratio that has already been added to noisy input

### Defining structures of training and testing models and their components
*Note that the trainnig model will be used further, however, the testting model is used only as a reference for its structural properties. The actual testing models will be generated dynamically.*

In [ ]:
# Models structures
keep_encoder=False
train_qae_qc, in_params, enc_params, dec_params, all_weight_params = \
    train_qae(num_latent, num_trash, reps=reps, ent=ent, in_seq_label='N', in_seq_name='Noisy Input', keep_encoder=keep_encoder)

train_qae_qc = train_qae_qc.decompose()
print(f'Encoder characteristics: \n\n'+
      f'\ttraining windows={len(y_train_enc)}\n'+
      f'\tvalidation windows={len(y_valid_enc)}\n'+
      f'\tinputs={len(in_params)}\n'+
      f'\tkeep_encoder={keep_encoder}\n'+
      f'\tweights={len(all_weight_params)}\n'+
      f'\ttrash layer={num_trash}\n'+
      f'\tlatent layer={num_latent}\n'+
      f'\ttraining qubits={len(train_qae_qc.qubits)}\n'+
      f'\ttesting qubits={len(train_qae_qc.qubits)}\n'+
      f'\treps={reps}\n'+
      f'\tent={ent}\n'+
      f'\tnoise={noise}\n'+
      f'\n'+
      f'\tcost type={cost_type}\n'+
      f'\tcost yscale={cost_yscale}\n'+
      f'\tsamples={y_train_enc.shape[0]}\n'+
      f'\tdevice={device}\n'+
      f'\tepochs={epochs}\n'+
      ''
     )

# print(f'\nTraining autoencoder:')
# print(f'\nData parameters:\n{in_params}')
# print(f'\nEncoder parameters:\n{enc_params}')
# if not keep_encoder:
#     print(f'\nDecoder parameters:\n{dec_params}')
# print(f'\nAll training parameters:\n{all_weight_params}')

print('')
display(train_qae_qc.draw("latex", style="iqp", scale=2))
print('')

### Preparing data for full-QAE training

In [ ]:
### Preparing data for training
#   - First half of each row is to fill in the noisy input block
#   - Second half of each row is to fill the expected pure output block

# Testing full-QAE with noise, but no separate Decoder
y_train_pure_and_noisy = []
for (y_noisy, y_pure) in zip(y_train_noisy_enc, y_train_enc):
    y_train_pure_and_noisy.append(list(y_noisy)+list(y_pure))
y_train_pure_and_noisy = np.array(y_train_pure_and_noisy)

print(f'\nNoisy training data:\n{y_train_noisy_enc[0:3]}')
print(f'\nPure training data:\n{y_train_enc[0:3]}')
print(f'\nNoisy+pure training data:\n{y_train_pure_and_noisy[0:3]}\n')

In [ ]:
### Testing paramers assignment

p_dict = []
p_no = 0
for p in in_params: #y_train_pure_and_noisy:
    p_dict.append(y_train_pure_and_noisy[0][p_no])
    # print(f'{p}={p_dict[p_no]}')
    p_no += 1
# print()

init_pt = algorithm_globals.random.random(len(all_weight_params))
p_no = 0
for p in all_weight_params:
    p_dict.append(init_pt[p_no])
    # print(f'{p}={p_dict[p_no]}')
    p_no += 1
# print()

qc=train_qae_qc
qc=qc.assign_parameters(p_dict)
qc.draw("latex", style="clifford", scale=1, fold=-1) # style=clifford/iqp

### File names to be used for saving results
Files to store modelinitialisation parameters, original input data, measured input data, QAE recovered data

In [ ]:
SAVE_PATH = 'results'
init_file_name = f'qae_wangle_lat{num_latent}_tr{num_trash}_reps{reps}_{ent}_p{len(all_weight_params)}_err{noise}_{cost_type}_init.json'
inorg_file_name = f'qae_wangle_lat{num_latent}_tr{num_trash}_reps{reps}_{ent}_p{len(all_weight_params)}_err{noise}_{cost_type}_org.json'
inmeas_file_name = f'qae_wangle_lat{num_latent}_tr{num_trash}_reps{reps}_{ent}_p{len(all_weight_params)}_err{noise}_{cost_type}_inm.json'
outrec_file_name = f'qae_wangle_lat{num_latent}_tr{num_trash}_reps{reps}_{ent}_p{len(all_weight_params)}_err{noise}_{cost_type}_out.json'
history_file_name = f'qae_wangle_lat{num_latent}_tr{num_trash}_reps{reps}_{ent}_p{len(all_weight_params)}_err{noise}_{cost_type}_hist{epochs}.json'
history_file_name

### Setting the initial point
Since model training may take a long time we have already pre-trained the model for some iterations and saved the pre-trained weights. We'll continue training from that point by setting `initial_point` to a vector of pre-trained weights.

In [ ]:
saved_starting_point = False

if saved_starting_point:
    ### Using previously saved init point
    print(f'Using previously saved init parameters from file "{SAVE_PATH}/{init_file_name}"')

    try:
        f = open(f'{SAVE_PATH}/{init_file_name}', 'r')
    except OSError:
        print(f'Could not open/read file: {SAVE_PATH}/{init_file_name}')
        sys.exit()
    with f:
        initial_point = json.load(f)
        history_file_name = f'{history_file_name[:-5]}_hist{epochs}.json'
        print(f'New history file will be created: "{SAVE_PATH}/{history_file_name}"')
else:
    ### Using a random init point
    print(f'Random init parameters to be saved to file "{SAVE_PATH}/{init_file_name}"')
    initial_point = algorithm_globals.random.random(len(all_weight_params))
    print(f'History file will be created: "{SAVE_PATH}/{history_file_name}"')

By minimizing this cost function, we can thus determine the required parameters to compress our noisy series.

### QAE training

In [ ]:
### Utility

def identity_interpret(x):
    return x

first_only = lambda x: x % 2

In [ ]:
### Test

sample_qnn = SamplerQNN(
    circuit=train_qae_qc,
    input_params=in_params,
    weight_params=all_weight_params,
    interpret=identity_interpret,
    output_shape=2**train_qae_qc.num_qubits,
    sampler=sampler
)
sample_data = y_train_pure_and_noisy[0:5]

start = time.time()
probs = sample_qnn.forward(sample_data, initial_point)
elapsed = time.time() - start
print(f'Forward pass result for SamplerQNN:\n{probs}.  \nShape: {probs.shape}, Time: {elapsed}')
print(f'Cost = 1-P(|0>^n) = {1-np.sum(probs[:, 0]) / sample_data.shape[0]} (to be minimised)')

In [ ]:
training_required = True

if training_required:
    
    print('Training initiated, optimum parameters will be calculated')
    
    qnn = SamplerQNN(
        circuit=train_qae_qc,
        input_params=in_params,
        weight_params=all_weight_params,
        interpret=identity_interpret,
        output_shape=2**train_qae_qc.num_qubits,
        sampler=sampler
    )
    
    start = time.time()
    
    opt = COBYLA(maxiter=epochs) # COBYLA, NELDER_MEAD, SPSA
    cost_func_tswind = Cost(y_train_pure_and_noisy, opt, initial_point, shuffle=True, 
                            cost_type=cost_type, yscale=cost_yscale)
    opt_result = cost_func_tswind.optimize()
    optimum_parameters = opt_result.x
    minimum_cost = opt_result.fun
    elapsed = time.time() - start

    # Print run statistics
    print(f'{device} Fit ('+
          f'{y_train_pure_and_noisy.shape[0]} samples, '+
          f'{reps} reps, '+
          f'"{ent}" ent, '+
          f'{num_latent}+{num_trash} qubits, '+
          f'{len(all_weight_params)} params, '+
          f'{epochs} epochs): '+
          f'{round(minimum_cost, 5)} min cost, '+
          f'{elapsed:0.2f} sec')

elif saved_starting_point:
    optimum_parameters = initial_point
    print('Training skipped, previously saved optimum parameters will be used here')
else:
    optimum_parameters = initial_point
    print('Training skipped, optimum parameters were generated randomly')    

## Model testing
Note that all components of a test circuit will be built in the testing process

In [ ]:
### Select a backend

backend = Aer.get_backend('aer_simulator_statevector')
# backend = Aer.get_backend('aer_simulator')

### QAE calculation and analysis of the selected TS windows

In [ ]:
### Select a collection of windows to test

# sel_wind_set = list(range(30))
# sel_wind_set = [1, 2, 3, 4, 5, 10, 20, 21, 22, 23, 30]
# sel_wind_set = [*range(0, 10)]+[*range(30, 50, 2)]+[*range(60, 75, 2)]
# sel_wind_set = [*range(0, y_valid_enc.shape[0], 3)]
# sel_wind_set = [*range(50, 100, 2)]
# sel_wind_set = [*range(50, 100, 1)]
# sel_wind_set = [*range(50)]
sel_wind_train_set = [*range(1, 20, 2)]
sel_wind_valid_set = [*range(len(y_valid_enc))]

print(f'Selected {len(sel_wind_train_set)} training windows to process')
print(f'Selected {len(sel_wind_valid_set)} validation windows to process')

In [ ]:
### Generation of QAE results for the selected windows

### Analyses a sample of windows, QAE(in) -> recons
### winds_pure: a list/array of TS windows
### sel_wind_set: a list of the selected windows indeces
### returns: dictionaries of a subset of original, measured, and reconstructured windows
###          the keys correspond to windows sequence position in a TS

def qae_analyse_sample(sel_wind_set, winds_pure, winds_noise=None):
    print(f'Processed windows: ', end=' ')

    in_org_set = {}
    in_meas_set = {}
    out_reconstr_set = {}

    for wind_idx in range(len(sel_wind_set)):
    
        sel_wind = sel_wind_set[wind_idx]
        if winds_noise is None:
            wind = winds_pure[sel_wind] # Noisy data was not provided so use pure-to-pure analysis
            in_org_set[sel_wind] = list(wind)
        else:
            wind = winds_noise[sel_wind] # Use noisy window for manipulation
            in_org_set[sel_wind] = list(winds_pure[sel_wind]) # But add pure window for reference
    
        ### Find the QAE input window state vector representation (as measured)
        org_meas = []
        for meas_q in range(len(wind)):
            org_qc, _ = sequence_encoder(num_latent + num_trash, wind_size, classreg=True, meas_q=meas_q)
            org_qc = org_qc.assign_parameters(wind)
            meas_angle = single_qubit_angle_meas(org_qc, backend, shots=10000)
            org_meas.append(meas_angle)
    
        ### Find the QAE resulting window state vector representation
        out_meas = []
        for meas_q in range(len(wind)):
            param_values = np.concatenate((wind, optimum_parameters))
    
            out_qc, _, _, _ = qae(num_latent, num_trash, reps=reps, ent=ent, classreg=True, keep_encoder=False, meas_q=meas_q)     
            out_qc = out_qc.assign_parameters(param_values)
            out_qc = out_qc.decompose().decompose()
            meas_angle = single_qubit_angle_meas(out_qc, backend, shots=10000)
            out_meas.append(meas_angle)
    
        ### Add window results
        in_meas_set[sel_wind] = org_meas
        out_reconstr_set[sel_wind] = out_meas    
    
        print(f'{sel_wind}', end=' ')
    
    print('End\n')
    return in_org_set, in_meas_set, out_reconstr_set

# load_qae_results = False

# if load_qae_results:

#     ### Loading previously saved original window data, measured input and reconstructed data

#     try:
#         inorg_file = open(inorg_file_name, 'r')
#         inmeas_file = open(inmeas_file_name, 'r')
#         outrec_file = open(outrec_file_name, 'r')
#     except OSError:
#         print(f'Could not open/read results files')
#         sys.exit()
#     with inorg_file, inmeas_file, outrec_file:
#         in_org_set = json.load(inorg_file)
#         in_meas_set = json.load(inmeas_file)
#         out_reconstr_set = json.load(outrec_file)

#     print(f'Loaded windows: {" ".join(in_org_set.keys())} End')



In [ ]:
### Analyse samples of training and validation data
in_org_train_set, in_meas_train_set, out_reconstr_train_set = \
    qae_analyse_sample(sel_wind_train_set, y_train_enc, y_train_noisy_enc) # Noise vs Pure
in_org_valid_set, in_meas_valid_set, out_reconstr_valid_set = \
    qae_analyse_sample(sel_wind_valid_set, y_valid_enc, y_valid_noisy_enc) # Noise vs Pure

### Reporting of QAE accuracy
Several model performance metrics can be calculated based on the selection of TS windows.<br/>
In the left column we have metrics comparing QAE inputs with the same but quantum measured inputs.<br/>
In the right column we have metrics comparing QAE inputs with its outputs - this is what we are intereted in.<br/>
The left column is for your reference to see the impact of measurement on the QAE precision.

In [ ]:
def accuracy_report(in_org_set, out_reconstr_set, in_meas_set, title='Precision of Results',
                   trim_left = 0, trim_right = 0):
    
    print(f'\n{title}\n')

    r2_inorg_incalc = r2_tswin(in_org_set, in_meas_set, trim_left=trim_left, trim_right=trim_right)
    r2_inorg_outrec = r2_tswin(in_org_set, out_reconstr_set, trim_left=trim_left, trim_right=trim_right)
    print(f'\tR2(in_org, in_calc) =\t{np.round(r2_inorg_incalc, 3)}, '+
          f'\tR2(in_org, out_rec) =\t{np.round(r2_inorg_outrec, 3)}')

    rms_inorg_incalc = rms_tswin(in_org_set, in_meas_set, trim_left=trim_left, trim_right=trim_right)
    rms_inorg_outrec = rms_tswin(in_org_set, out_reconstr_set, trim_left=trim_left, trim_right=trim_right)
    print(f'\tRMS(in_org, in_calc) =\t{np.round(rms_inorg_incalc, 3)}, '+
          f'\tRMS(in_org, out_rec) =\t{np.round(rms_inorg_outrec, 3)}')

    mae_inorg_incalc = mae_tswin(in_org_set, in_meas_set, trim_left=trim_left, trim_right=trim_right)
    mae_inorg_outrec = mae_tswin(in_org_set, out_reconstr_set, trim_left=trim_left, trim_right=trim_right)
    print(f'\tMAE(in_org, in_calc) =\t{np.round(mae_inorg_incalc, 3)}, '+
          f'\tMAE(in_org, out_rec) =\t{np.round(mae_inorg_outrec, 3)}')

    mape_inorg_incalc = mape_tswin(in_org_set, in_meas_set, trim_left=trim_left, trim_right=trim_right)
    mape_inorg_outrec = mape_tswin(in_org_set, out_reconstr_set, trim_left=trim_left, trim_right=trim_right)
    print(f'\tMAPE(in_org, in_calc) =\t{np.round(mape_inorg_incalc, 3)}, '+
          f'\tMAPE(in_org, out_rec) =\t{np.round(mape_inorg_outrec, 3)}\n')

    return r2_inorg_incalc, r2_inorg_outrec

In [ ]:
r2_inorg_train_incalc, r2_inorg_train_outrec = \
    accuracy_report(in_org_train_set, in_meas_train_set, out_reconstr_train_set, title='Training Accuracy')
r2_inorg_valid_incalc, r2_inorg_valid_outrec = \
    accuracy_report(in_org_valid_set, in_meas_valid_set, out_reconstr_valid_set, title='Validation Accuracy')

### Ploting sample QAE results
Note the edge distortion, which occurs either at the window's beginning, or more commonly at the end, or both.

In [ ]:
### Plot all the selected windows and their recovered data

def qae_plot_winds(in_org_set, in_meas_set, out_reconstr_set, plot_no=None):
    
    sorted_keys = sorted(in_org_set.keys())
    if plot_no == None:
        plot_no = len(sorted_keys)
        
    for sel_wind in sorted_keys:

        if plot_no == 0:
            break
    
        # Retrieve window data
        print('\n')
        wind = in_org_set[sel_wind]
        org_meas = in_meas_set[sel_wind]
        out_meas = out_reconstr_set[sel_wind]    
        
        # Plot prepared data
        plt.rcParams["figure.figsize"] = (12, 6)
        plt.title(f'Original and measured input vs reconstructed data, within window {sel_wind}')
        plt.xlabel("Data points")
        plt.ylabel("Inter-point differences")
        # plt.xlim(lb, ub)
        
        # Plot target function
        plt.plot([x for x in range(len(wind))], wind, color='royalblue', label='Original data')
        plt.plot([x for x in range(len(wind))], wind, marker='o', color='cornflowerblue', linestyle='None')
        plt.plot([x for x in range(len(wind))], org_meas, color='darkorange', label='Measured data')
        plt.plot([x for x in range(len(wind))], org_meas, marker='o', color='orange', linestyle='None')
        plt.plot([x for x in range(len(wind))], out_meas, color='red', label='Reconstructed data')
        plt.plot([x for x in range(len(wind))], out_meas, marker='o', color='tomato', linestyle='None')
        plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.2),
                  ncol=3, fancybox=True, shadow=True)
        plt.show()

        plot_no -= 1
        
    print('\n')

#### Testing noise to noise - can we believe what we see!
If the red reconstructed curve follow the blue noise curve given on input then we have a problem!<br/>
The yellow control curve, which just measures the noisy data, should follow the provided noisy input.<br/>
If the red curve is consistently different from both the blue and yellow lines, then it is all great,<br/>
as it means the full-QAE has learnt not to follow what was given on input!

In [ ]:
### We pretend that the noisy data is pure, so blue and yellow lines follow noise
in_org_valid_noisy_set, in_meas_valid_noisy_set, out_reconstr_valid_noisy_set = \
    qae_analyse_sample(sel_wind_valid_set, y_valid_noisy_enc) # Noise vs Pure
qae_plot_winds(in_org_valid_noisy_set, in_meas_valid_noisy_set, out_reconstr_valid_noisy_set, plot_no=10)

#### Plotting results of analysis performed on training data (previously seen by the full-QAE)
The blue line shows pure data that was given on output during training.</br>
The red curve was reconstructed from the noisy data given on input (not shown here).<br/>
The yellow control curve measures the pure data, it follows the blue curve, the best it can.<br/>
The red curve should follow the blue pure data and its variance should not be much different from the yellow lines,<br/>
as it means it has learnt to recover pure data from noise given on input!<br/>
These charts only show that the model has good memory of data shown in training, subject to measurement error.

In [ ]:
qae_plot_winds(in_org_train_set, in_meas_train_set, out_reconstr_train_set, plot_no=10)

#### Plotting results of analysis performed on validation data (not seen by the full-QAE)
The blue line shows pure data that was given on output during training.</br>
The red curve was reconstructed from the noisy data given on input (not shown here).<br/>
The yellow control curve measures the pure data, it follows the blue curve, the best it can.<br/>
The red curve should follow the blue pure data and its variance should not be much different from the yellow lines,<br/>
as it means it has learnt to recover pure data from noise given on input!<br/>
These charts show that the model can generalise away from data shown in training, subject to measurement error.

In [ ]:
qae_plot_winds(in_org_valid_set, in_meas_valid_set, out_reconstr_valid_set, plot_no=10)

### Accuracy of results after manual removal of edge distortion in a series windows
Based on visual inspection of the QAE results, we can observe that the QAE results are distored at the edges.<br/>
So it is possibe to exclude a few data points from each data window, and hence improve the model accuracy.<br/><br/>
In the left column we have metrics comparing QAE inputs with the same but quantum measured inputs.<br/>
In the right column we have metrics comparing QAE inputs with its outputs - this is what we are intereted in.<br/>
The left column is for your reference to see the impact of measurement on the QAE precision.

In [ ]:
### Present some statistics on the sample, after removing the edge distortion
trim_left = 0
trim_right = 2

r2_adj_inorg_train_incalc, r2_adj_inorg_train_outrec = accuracy_report(in_org_train_set, in_meas_train_set, out_reconstr_train_set, 
                title='Training Accuracy', trim_left=trim_left, trim_right=trim_right)
r2_adj_inorg_valid_incalc, r2_adj_inorg_valid_outrec = accuracy_report(in_org_valid_set, in_meas_valid_set, out_reconstr_valid_set, 
                title='Validation Accuracy', trim_left=trim_left, trim_right=trim_right)

### Saving the starting point and calculated results

In [ ]:
print(f'Files to be saved (when True)\n')
print(f'Optimisation init point: {SAVE_PATH}/{init_file_name}')
print(f'Original test windows:   {SAVE_PATH}/{inorg_file_name}')
print(f'Measured test windows:   {SAVE_PATH}/{inmeas_file_name}')
print(f'Resulting test windows:  {SAVE_PATH}/{outrec_file_name}\n')

In [ ]:
### Save the initial point, also save the history file
to_be_saved_starting_point = True

if to_be_saved_starting_point:
    ## Saving the initialisation point
    with open(f'{SAVE_PATH}/{init_file_name}', 'w') as f:
        json.dump(opt_result.x.tolist(), f)
        print(f'Saved optimum parameters as the future starting point in:\n\t{init_file_name}')

    ### Saving the history file
    with open(f'{SAVE_PATH}/{history_file_name}', 'w') as h:
        json.dump(cost_func_tswind.objective_func_vals, h)
        print(f'Saved optimisation history in:\n\t{history_file_name}')
else:
    print(f'Parameters and history not saved') 

In [ ]:
### Save all calculated data points
to_be_saved_results = True

if to_be_saved_results:
    ## Saving results
    with open(f'{SAVE_PATH}/{inorg_file_name}', 'w') as f:
        json.dump(in_org_valid_set, f)
    with open(f'{SAVE_PATH}/{inmeas_file_name}', 'w') as f:
        json.dump(in_meas_valid_set, f)
    with open(f'{SAVE_PATH}/{outrec_file_name}', 'w') as f:
        json.dump(out_reconstr_valid_set, f)
    print(f'Saved results in:\n\t{inorg_file_name}\n\t{inmeas_file_name}\n\t{outrec_file_name}')
else:
    print(f'Sample windows not saved') 

## Time series integration and plotting
The aim is to merge all windows produced in testing into a single sequence, stored as a dictionary. As depicted in the following figure, the integration procedure needs to average all overalapping windows values (white boxes), while considering the step between windows (which creates gaps between overlapping windows) and their edge trimming (which removes distorted values, indicated as gray boxes). 

<div>
    <!--img src="attachment:wind-integration.png" width="1000"-->
    <img src="../images/wind-integration.png" width="1000">
</div>

**Figure: Windows integration procedure**

### Windows integration

In [ ]:
### Integrates QAE results stored in a single set of TS widows into a single sequence
#   Note: When windows overap their average values will be returned
#         When windows are too far apart, separate sub-sequences will be returned
#   wind_set: The selected set of TS windows
#   trim_left: The number of values to be trimmed from the left edge of each window
#   trim_right:  The number of values to be trimmed from the right edge of each window

def qae_winds_integ_1(wind_set, trim_left=0, trim_right=0):

    # Collect all overalapping values into lists attached to individual data points
    vals = {}
    for k in sorted(wind_set.keys()):
        val_list = wind_set[k]
        list_len = len(val_list)
        for i in range(list_len):
            if (i < trim_left) or (i >= list_len - trim_right):
                None # Skip these trimmed values
            else:
                val_idx = k+i
                if val_idx in vals:
                    vals[val_idx].append(val_list[i])
                else:
                    vals[val_idx] = [val_list[i]]

    # Collapse all consecutive values into subsequences
    # All values apart start new subsequences
    seq = {}
    next_key = -3
    prev_key = 0
    for k in sorted(vals.keys()):
        next_key += 1
        if k == next_key:
            seq[prev_key].append(np.average(vals[k]))
        else:
            prev_key = k
            next_key = k
            seq[k] = [np.average(vals[k])]
            
    return seq


### Integrates QAE results stored as TS widows into a single sequence
#   Note: When windows overap their average values will be returned
#         When windows are too far apart, separate sub-sequences will be returned
#   in_org_set: The selected set of original TS windows
#   in_meas_set: The set of measurements of the original values in TS windows
#   out_recons_set: The set of QAE reconstructions for each original TS window
#   trim_left: The number of values to be trimmed from the left edge of each window
#   trim_right:  The number of values to be trimmed from the right edge of each window

def qae_winds_integ(in_org_set, in_meas_set, out_recons_set, trim_left=0, trim_right=0):
    in_org_seq = qae_winds_integ_1(in_org_set, trim_left, trim_right)
    in_meas_seq = qae_winds_integ_1(in_meas_set, trim_left, trim_right)
    out_recons_seq = qae_winds_integ_1(out_recons_set, trim_left, trim_right)
    return in_org_seq, in_meas_seq, out_recons_seq

### Test window integration
# in_org_seq, in_meas_seq, out_recons_seq = qae_winds_integ(in_org_set, in_meas_set, out_reconstr_set, trim_left, trim_right)
# [print((k, out_reconstr_set[k])) for k in sorted(out_reconstr_set.keys())]
# print()
# vals = qae_winds_integ_1(out_recons_seq)
# [print((k, vals[k])) for k in sorted(vals.keys())]

### Integrated windows plotting

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

### Returns the x range for the sequence set
def qae_seq_x_range(start, subseq):
    return range(start, start+len(subseq))

### Plots all three sequences (may consist of separate subsets)
def qae_seq_plot(in_org_seq, in_meas_seq, out_recons_seq, 
                 add_markers=False,
                 label_suffix=['', '', ''],
                 xlabel='Selected data points', ylabel='Inter-point differences',
                 title=f'Original and measured input vs reconstructed data'):
    
    sorted_keys = sorted(in_org_seq.keys())
    
    # Plot prepared data
    plt.rcParams["figure.figsize"] = (12, 6)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    # plt.xlim(lb, ub)
        
    for sel_wind in sorted_keys:

        org_in = in_org_seq[sel_wind]
        org_meas = in_meas_seq[sel_wind]
        out_meas = out_recons_seq[sel_wind]    
        
        # Plot target function
        plt.plot(qae_seq_x_range(sel_wind, org_in), org_in, color='royalblue')
        if add_markers:
            plt.plot(qae_seq_x_range(sel_wind, org_in), org_in, marker='o', color='cornflowerblue', linestyle='None')
        plt.plot(qae_seq_x_range(sel_wind, org_meas), org_meas, color='darkorange')
        if add_markers:
            plt.plot(qae_seq_x_range(sel_wind, org_meas), org_meas, marker='o', color='orange', linestyle='None')
        plt.plot(qae_seq_x_range(sel_wind, out_meas), out_meas, color='red')
        if add_markers:
            plt.plot(qae_seq_x_range(sel_wind, out_meas), out_meas, marker='o', color='tomato', linestyle='None')
        
    # access legend objects automatically created from data
    handles, labels = plt.gca().get_legend_handles_labels()
    
    # create manual symbols for legend
    line_org = Line2D([0], [0], label=f'Original data {label_suffix[0]}', color='royalblue')
    line_meas = Line2D([0], [0], label=f'Measured data {label_suffix[1]}', color='darkorange')
    line_recon = Line2D([0], [0], label=f'Reconstructed data {label_suffix[2]}', color='red')
    
    # add manual symbols to auto legend
    handles.extend([line_org, line_meas, line_recon])
    
    # plt.legend(handles=handles)    
    plt.legend(handles=handles, loc='lower center', bbox_to_anchor=(0.5, -0.2),
               ncol=3, fancybox=True, shadow=True)
    plt.show()

    print('\n')
    return
    

In [ ]:
# Plot all raw results
in_org_train_seq, in_meas_train_seq, out_recons_train_seq = qae_winds_integ(in_org_train_set, in_meas_train_set, out_reconstr_train_set)
qae_seq_plot(in_org_train_seq, in_meas_train_seq, out_recons_train_seq, title=f'Original and measured input vs reconstructed training data (with averaged raw results)',
             label_suffix=[f'', f'(R2: {np.round(r2_inorg_valid_incalc, 3)})', f'(R2: {np.round(r2_inorg_valid_outrec, 3)})'])

in_org_valid_seq, in_meas_valid_seq, out_recons_valid_seq = qae_winds_integ(in_org_valid_set, in_meas_valid_set, out_reconstr_valid_set)
qae_seq_plot(in_org_valid_seq, in_meas_valid_seq, out_recons_valid_seq, title=f'Original and measured input vs reconstructed validation data (with averaged raw results)',
             label_suffix=[f'', f'(R2: {np.round(r2_inorg_valid_incalc, 3)})', f'(R2: {np.round(r2_inorg_valid_outrec, 3)})'])

In [ ]:
# Plot all results with distortion trimming
in_org_adj_train_seq, in_meas_adj_train_seq, out_recons_adj_train_seq = qae_winds_integ(in_org_train_set, in_meas_train_set, out_reconstr_train_set, trim_left, trim_right)
qae_seq_plot(in_org_adj_train_seq, in_meas_adj_train_seq, out_recons_adj_train_seq, title=f'Original and measured input vs reconstructed training data (with edge distortion removed)',
             label_suffix=[f'', f'(R2: {np.round(r2_adj_inorg_valid_incalc, 3)})', f'(R2: {np.round(r2_adj_inorg_valid_outrec, 3)})'])

in_org_adj_valid_seq, in_meas_adj_valid_seq, out_recons_adj_valid_seq = qae_winds_integ(in_org_valid_set, in_meas_valid_set, out_reconstr_valid_set, trim_left, trim_right)
qae_seq_plot(in_org_adj_valid_seq, in_meas_adj_valid_seq, out_recons_adj_valid_seq, title=f'Original and measured input vs reconstructed validation data (with edge distortion removed)',
             label_suffix=[f'', f'(R2: {np.round(r2_adj_inorg_valid_incalc, 3)})', f'(R2: {np.round(r2_adj_inorg_valid_outrec, 3)})'])

## References

1. A wikipedia page on Autoencoder: https://en.wikipedia.org/wiki/Autoencoder

2. Romero, Jonathan, Jonathan P. Olson, and Alan Aspuru-Guzik. "Quantum autoencoders for efficient compression of quantum data." Quantum Science and Technology 2.4 (2017): 045001.

3. Swap Test Algorithm: https://en.wikipedia.org/wiki/Swap_test

4. Bravo-Prieto, Carlos, "Quantum autoencoders with enhanced data encoding." Machine Learning: Science and Technology, 2, May 2021

## Performance Statistics
The format of the presented statistics was evolving over time. In the current edition, which is most evident in the reporting for the experiments running on the "furies-vengence" machine, the reported details include:
- device: Device used to accelerate computation (GPU/CPU)
- samples: Number of samples used in training (numeric)
- reps: Number of repeated blocks in ansatz (numeric)
- ent: Entaglement type (linear, reverse_linear, full, circular, sca, pairwise)
- qubits l+t: Number of core ansatz qubits used in QAE testing (numeric latent + trash qubits)
- params: Number of trainable parameters (numeric)
- epoch: Number of training iterations (numeric)
- min cost: Minimum cost achieved in circuit optimisation (numeric)
- sec: Elapsed time in seconds used in circuit training (numeric)
- R2: R squared providing goodness of fit between the TS window and its reconstruction (0..1)
- Trim l+r: R2 measure between windows trimmed to deal with edge distortion (numeric left + right)
- comment: Additional remark on the experiment's success (e.g. "Promissing" or "Disappointing")
- colour: Colour siginfying the "goodness" of selected experiment's controls (blue=great, red=terrible, magenta=note this, black=FYI)

### furies-vengence: ASUS 64Gb RAM
- CPU Intel i9, 24 cores, 32 threads, 2.2-3.0-4.3-5.4GHz
- GPU NVIDIA GTX 4090, 16384 CUDA Cores, 24 Gb RAM, CUDA V12.3, NVIDIA Driver 545.29.06

**Half-QAE Training**

<font color="blue">GPU Fit (275 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 300 epochs):</font> **0.00301 min cost, 98.32 sec** (R2=0.815, Trim 0+2 R2=0.984, Promising)<br/>
<font color="blue">GPU Fit (275 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs):</font> **0.00305 min cost, 65.55 sec** (R2=0.838, Trim 0+2 R2=0.986, Promising)<br/>
<font color="green">GPU Fit (275 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs):</font> **0.03907 min cost, 57.87 sec** (R2=0.764, Trim 0+2 R2=0.967, Test Cost=1-P(|0>^t, no swap))<br/>
<font color="magenta">CPU Fit (275 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs):</font> **0.00379 min cost, 146.40 sec** (R2=0.785, Trim 0+2 R2=0.976, CPU is 0.45 of GPU speed)<br/>
<font color="blue">GPU Fit (274 samples, 4 reps, "sca" ent, 8+2 qubits, 50 params, 600 epochs):</font> **0.0039 min cost, 236.92 sec** (R2=0.745, Trim 1+1 R2=0.975, Promising)<br/>

**Full-QAE Training**

GPU Fit (135 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs): **0.02601 min cost, 6582.02 sec** (Train: Pure->Pure, Enc/Enc-dg only, Opt to Zero)<br/>
&emsp;&emsp;Match: Pure vs Pure (R2=0.707, Trim 0+2 R2=0.923)<br/>

GPU Fit (135 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs): **0.03173 min cost, 6689.22 sec** (Train: Noise->Pure, Enc/Enc-dg only, Opt to Zero)<br/>
&emsp;&emsp;Match: Pure vs Pure (R2=0.768, Trim 0+2 R2=0.989) / Noise vs Noise (R2=0.767, Trim 0+2 R2=0.988) / Noise vs Pure (R2=0.563, Trim 0+2 R2=0.721)<br/>

GPU Fit (135 samples, 2 reps, "sca" ent, 5+3 qubits, 24 params, 200 epochs): **0.03461 min cost, 6770.68 sec** (Train: Noise->Pure, Enc/Enc-dg only, Opt to Zero)<br/>
&emsp;&emsp;Match: Pure vs Pure (R2=0.653, Trim 0+2 R2=0.841) / Noise vs Noise (R2=0.658, Trim 0+2 R2=0.843) / Noise vs Pure (R2=0.509, Trim 0+2 R2=0.66)<br/>

GPU Fit (135 samples, 2 reps, "sca" ent, 4+4 qubits, 24 params, 200 epochs): **0.04668 min cost, 7057.43 sec** (Train: Noise->Pure, Enc/Enc-dg only, Opt to Zero)<br/>

GPU Fit (135 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs): **0.06584 min cost, 3874.71 sec** (Train: Noise->Pure, Enc/Enc-dg only, Opt to P(|0>^8))<br/>
&emsp;&emsp;Match: Pure vs Pure (R2=0.766, Trim 0+2 R2=0.995) / Noise vs Noise (R2=0.774, Trim 0+2 R2=0.994) / Noise vs Pure (R2=0.536, Trim 0+2 R2=0.698)<br/>

GPU Fit (135 samples, 2 reps, "sca" ent, 6+2 qubits, 48 params, 200 epochs): **0.15521 min cost, 3592.66 sec** (Train: Noise->Pure, Enc/Dec-dg, Opt to P(|0>^8))<br/>
&emsp;&emsp;Epochs 1-200 epocha: Pure vs Pure (R2=0.297, Trim 0+3 R2=0.553) / Noise vs Noise (R2=0.36, Trim 0+3 R2=0.612) / Noise vs Pure (R2=0.165, Trim 0+3 R2=0.398)<br/>
GPU Fit (135 samples, 2 reps, "sca" ent, 6+2 qubits, 48 params, 200 epochs): **0.08664 min cost, 3891.44 sec** (Train: Noise->Pure, Enc/Dec-dg, Opt to P(|0>^8))<br/>
&emsp;&emsp;Epochs 201-400 epocha: Pure vs Pure (R2=0.696, Trim 0+3 R2=0.906) / Noise vs Noise (R2=0.702, Trim 0+3 R2=0.913) / Noise vs Pure (R2=0.558, Trim 0+3 R2=0.742)<br/>

<font color="blue">GPU Fit (29 samples, 2 reps, "sca" ent, 6+2 qubits, 48 params, 500 epochs):</font> **0.05421 min cost, 2116.93 sec**  (Train: Noise->Pure, Enc/Dec-dg, Opt to P(|0>^8))</br>
&emsp;&emsp;training: Noise vs Pure (R2=0.713, Trim 0+2 R2=0.609), validation: Noise vs Pure (R2=0.703, Trim 0+2 R2=0.722, Promising)<br/>


**Earlier Half-QAE Training**

GPU Fit (275 samples, 2 reps, "linear" ent, 6+2 qubits, 24 params, 200 epochs): **0.00884 min cost, 65.01 sec** (R2=404, Trim R2=NA)<br/>
GPU Fit (275 samples, 2 reps, "circular" ent, 6+2 qubits, 24 params, 200 epochs): **0.014 min cost, 64.83 sec** (R2=-0.303, Trim R2=NA)<br/>
GPU Fit (275 samples, 2 reps, "pairwise" ent, 6+2 qubits, 24 params, 200 epochs): **0.00629 min cost, 54.41 sec** (R2=0.509, Trim 0+4 R2=0.986)<br/>
GPU Fit (275 samples, 2 reps, "full" ent, 6+2 qubits, 24 params, 200 epochs): **0.00556 min cost, 68.24 sec** (R2=0.716, Trim 0+3 R2=0.971)<br/>
GPU Fit (275 samples, 3 reps, "sca" ent, 6+2 qubits, 32 params, 300 epochs): **0.005 min cost, 101.94 sec** (R2=0.766, Trim 2 R2=0.98, Promising)<br/>
GPU Fit (274 samples, 2 reps, "sca" ent, 8+2 qubits, 30 params, 400 epochs): **0.00459 min cost, 140.45 sec** (R2=0.716, Trim 2 R2=0.76)<br/>
GPU Fit (274 samples, 3 reps, "sca" ent, 8+2 qubits, 40 params, 500 epochs): **0.00621 min cost, 182.33 sec** (R2=0.814, Trim 2 R2=0.872)<br/>
GPU Fit (274 samples, 4 reps, "sca" ent, 8+2 qubits, 50 params, 400 epochs): **0.00404 min cost, 157.36 sec** (R2=0.715, Trim 1+1 R2=0.971)<br/>
GPU Fit (274 samples, 4 reps, "circular" ent, 8+2 qubits, 50 params, 600 epochs): **0.01182 min cost, 243.13 sec** (R2=0.435, Trim R2=NA)<br/>
GPU Fit (274 samples, 4 reps, "pairwise" ent, 8+2 qubits, 50 params, 600 epochs): **0.00751 min cost, 224.47 sec** (R2=0.629, Trim 0+4 R2=0.857)<br/>


### basilisk: Lenovo 32Gb RAM
- CPU Intel i7, 4 cores, 8 threads, 2.8GHz - No GPU

CPU Fit (110 samples, 6+2 qubits, 32 params): **0.00744 min cost, 226.05 sec**<br/>
CPU Fit (275 samples, 6+2 qubits, 32 params, 400 epochs): **0.00512 min cost, 797.54 sec**<br/>
CPU Fit (275 samples, 6+2 qubits, 32 params, 250 epochs): **0.00623 min cost, 531.32 sec**<br/>
<font color="red">CPU Fit (275 samples, 6+2 qubits, 18 params, 250 epochs):</font> **0.00314 min cost, 308.64 sec** (Bravo-Prieto - terrible)<br/>
<font color="red">CPU Fit (275 samples, 6+2 qubits, 18 params, 125 epochs):</font> **0.00407 min cost, 155.59 sec** (Bravo-Prieto - terrible)<br/>
<font color="red">CPU Fit (275 samples, 6+2 qubits, 18 params, 125 epochs):</font> **0.00684 min cost, 157.55 sec** (Bravo-Prieto - terrible)<br/>
<font color="red">CPU Fit (275 samples, 6+2 qubits, 18 params, 125 epochs):</font> **0.00329 min cost, 154.95 sec** (Bravo-Prieto - terrible)<br/>
CPU Fit (275 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs): **0.00359 min cost, 284.36 sec** (R2=-6.629, Trim 0+3 R2=-1.783, Test 2)<br/>

### goblin-galore: Tomahawk 64Gb RAM
- CPU Intel i7, 6 cores, 12 threads, 3.5GHz - 2 GTX NVIDIA 1080 T1 GPUs, CUDA V12.2, NVIDIA Driver 535
- Note that this GPU at 6.1 compute capability is not compatible with Qiskit (cuQuantum requirement: +7 compute capability)

CPU Fit (275 samples, 6+2 qubits, 32 params, 400 epochs): **0.00456 min cost, 695.26 sec**<br/>
CPU Fit (274 samples, 8+2 qubits, 40 params, 400 epochs): **0.00364 min cost, 2430.50 sec** (Promising)<br/>
CPU Fit (274 samples, 6+3 qubits, 45 params, 500 epochs): **0.00907 min cost, 3603.48 sec** (Disappointing)<br/>
CPU Fit (275 samples, 5+3 qubits, 40 params, 300 epochs): **0.02233 min cost, 731.32 sec** (Disappointing)<br/>
CPU Fit (275 samples, 5+3 qubits, 40 params, 300+100 epochs): **0.01913 min cost, 245.12 sec** (Disappointing)<br/>
CPU Fit (275 samples, 6+2 qubits, 24 params, 300 epochs): **0.00334 min cost, 453.17 sec** (Promising, Test 0)<br/>
CPU Fit (275 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs): **0.00364 min cost, 309.03 sec** (R2=0.655, Trim 0+2 R2=0.934, Test 1 - Trig vs other Trig)<br/>
CPU Fit (275 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs): **0.00364 min cost, 313.68 sec** (R2=-11.302, Trim 0+3 R2=-1.033, Test 2a - Trig vs Poly)<br/>
CPU Fit (275 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs): **0.00014 min cost, 296.56 sec** (R2=-10.004, Trim 0+2 R2=-0.94, Test 2b - Poly vs Poly)<br/>
CPU Fit (275 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 160 epochs): **0.00044 min cost, 236.67 sec** (R2=-12.575, Trim 0+2 R2=-0.874, Test 2c - Poly vs Poly, short)<br/>
CPU Fit (275 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs): **0.00752 min cost, 305.49 sec** (R2=0.749, Trim 0+2 R2=0.992, Test 3 - Jitter vs Trig)<br/>
CPU Fit (275 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs): **0.00014 min cost, 306.52 sec** (R2=-8.721, Trim 0+2 R2=-1.014, Test 4a - Linear vs Linear)<br/>
CPU Fit (274 samples, 2 reps, "sca" ent, 8+2 qubits, 30 params, 200 epochs): **9e-05 min cost, 987.61 sec** (R2=-8.5, Trim 0+2 R2=-5.671, Test 4b - Linear vs Linear)<br/>
CPU Fit (275 samples, 2 reps, "sca" ent, 6+2 qubits, 24 params, 200 epochs): **0.01954 min cost, 305.30 sec** (R2=0.758, Trim 0+2 R2=0.954, Test 5 Beer)<br/>
CPU Fit (275 samples, 2 reps, "sca" ent, 5+3 qubits, 24 params, 200 epochs): **0.03795 min cost, 340.04 sec** (R2=0.575, Trim 0+2 R2=0.717, Test 6 Beer)<br/>
CPU Fit (275 samples, 2 reps, "sca" ent, 4+4 qubits, 24 params, 200 epochs): **0.06599 min cost, 872.53 sec** (R2=0.201, Trim 2+2 R2=0.349, Test Summary Beer)<br/>


## Logs

**Qiskit QAE with Unary Encoding**
- V1 Using QAE for anomaly detection (by denoising)
    - V1.1 Tested binary encoding, which misses proximity
    - V2.0 Created an "unary" encoding scheme
    - V2.1 Unary encoding tested with various options, not close
      - Issues noted:
          - Issue 01 (Completed) *Not sure how to interpret the output - cannot be visual*
          - Issue 02 (Completed) *Visual match between input and output not close*
          - Issue 03 (Completed) *Noise needs to be injected in training*
          - Issue 04 (Completed) *Training is getting slow, need to use GPU*
          - Issue 05 (Completed) *It is worth testing if an increase of the latent or trash layers could improve the result*
      - Action planned:
          - *Add Aer GPU*, *Add noise in training*, *Add convolution*, *Create classical solution to compare*
    - V2.2 Adding GPU plus encoding test
      - Encoding test:
          - Tested amplitude encoding. To improve the QAE we may need to focus on other aspects of the model,<br/>
            e.g. incorporation of noise and improvement to the ansatz to ensure neighbourhoods are strongly
            entangled.
      - Several problems installing GPU support, attempt to address Issue 04
          - Issue 06 (Completed) *Several problems associated with GPU support*<br/>
          - **Fix 06** *Need to install ML 0.7.1+ and Aer 0.13.2+*<br/>
            ML 0.7.1 standard installation (no need to install from source anymore)<br/>
            Aer 0.13.2 requires CUDA + cuQuantum + cuTensor (Deb)<br/>
            - Note that Aer Sampler has a bug and will not work with RawFeatureVector.
          - **Fix 04** *With GPU the training speed increased ~2 times
    - V2.3 Attempt to move to PuTorch, abandoned for the time being
    - V2.4 Changing to angle encoding - relative with Ry over 2pi
      - Issues noted:
          - Issue 07 (Completed) *Negative numbers not coded properly*</br>
      - Action planned:
          - *Change encoding as relative from H over pi* - with [-1:0] pi/2 negatives towards |0> and [0:+1] pi/2 positives towards |1>
    - V2.5 Completed QAE with angle encoding, lots of fixes
      - Fixes completed:
          - **Fix 07** *Adopted encoding around H state, up negative, down positive*
          - **Fix 02** *Resulting QAE outputs better match the inputs*
          - **Fix 01** *With the new encoding, the output is easy to interpret*
      - Issues noted:
          - *Issue 08* (Partially Completed) *QAE reconstruction has large variance end of windows*, averaging overlapping windows may help
          - Issue 09 (Completed) *Need to reconstruct the entire series*, possibly with averaging of overlapping windows
      - Action planned:
          - Issue 10 (Completed) *Systematically experiment with:* data sets, window size, step size, trash size, qubit no, ansatz reps and entanglement type
    - V2.6 Testing various QAE configurations
      - Varied the number of qubits (8 was best) and asatz reps (4 was best)
      - Issues noted:
          - Issue 11 (Completed) *Random seed seems to have little effect on results*
          - Issue 12 (Completed) *QAE results tend to be close to zero at the window ends, e.g. 1st and 7th value [0..7]*
          - Issue 13 (Completed) *Related to #12, the first window value should not be zero but the diff with the value from the previous window*<br/>
            Change the order of processing - first create deltas, then split into windows, change TS value encoding and decoding (no need to delta)
    - V2.7 Testing with an older computer Tomahawk, 64Gb RAM, 2 x GeForce GTX 1080 Ti, also testing the enhance encoding of Bravo-Prieto [4]
      - Issues noted:
          - Issue 14 (Completed) *GPU failed with error:*<br/>
            Simulation failed and returned the following error message:<br/>
            ERROR:  [Experiment 0] This program was not compiled for SM 61<br/>
          - Issue 15 (Completed) *A series of experiments with Bravo-Prieto's algorithm failed to produce any results*
      - Fixes completed:
          - **Fix 14** Reason (as provided on Slack), CuQuantum does not support GTX 1080 GPUs (cat.SM 61) - may not be fixable, GPU too old
          - **Fix 15** Bravo-Prieto algorithm could not be reproduced in any form, and with all possible options - it was abandoned*
      - CPU run works well - faster than Lenovo, results are pretty good (as per V2.6)
    - V2.8 Fixed incorrect coding of the first window value
      - Fixes completed:
          - **Fix 11** *Added **seed-simulator** to the backend options for GPU, this may be added for CPU when using AerSampler for CPU*
          - **Fix 12-13** *The first window encoded value is now delta of the first value minus the last value from the previous window*<br/>
            This was achieved by encoding the entire time series first, before splitting it into windows<br/>
            Additional charts were provided to show the original data, encoded sequence, and testing sequence<br/>
            QAE results at the edges of a window are non zero
      - Issues notes:
          - Issue 16 (Completed) *Noted high accuracy of results in the middle of windows, however, the edges are distorted* - related to Issue 08 and issue 12.<br/>
            The causes of this phenomenon may be: (1) the ansatz entanglement strategy, (2) position of Swap Test and recovery of the trash space, or (3) the sequential presentation of input.<br/>
            A possible approach would be to shuffle data on input, move the position of the trash to the middle, or to reject the values on edges.<bt/>
            To test this we need to experiments with larger windows
    - V2.9 Tested QAE with windows of size 10 - great results
      - Fixes completed:
          - Fix 08 (Completed) *When testing with windows of size 8 and 10, the edge inaccuracy persists but shifts*<br/>
            This may be the feature of the approach with Swap Test training
      - Action planned:
        This run was saved and the generated parameters and data will be used to address **Issue 9** (reconstruction of the series)
    - V2.10 Started a cleanup of all code
      - A test was conducted to create a QAE with 5/6 latent and 3 trash qubits. Its results were disappointing. The notebook, however, was copied to the Runs folder.
    - V2.11 Finished cleanup
    - V2.12 Adding precision stats of QAE results - R2, RMS, MAE and MAPE, with optional edge trimming of windows to increase precision
    - V2.13 Extendend the cost function, now part of the class "Cost"<br/>
      - Work conducted:
          - Implemented class "Cost", which allows parameterisation of the function behaviour.
          - Attempt 16 (Fully experimented) with the following results:
            - *Implemented shuffling of data durinug training, but there was no impact on the edge distortion*
            - *Tested variety of entanglement options, "sco" was found to be the best, other strategies produced distortions elsewhere and inferior results, so this may still be the culprit*
            - *Position of the trash space (and the Swap Test) seems to coincide with the edge distortion, so it could be its cause - needs a separate experiment*
    - V2.14 Merging all QAE test windows into a single time series (considering windows step and edge trimming)
      - Fixes completed:
        - **Fix 05** Experimented with different size of trash / latent area were conducted. with different outcomes, as reported.
        - **Fix 08, 16** The edge distortion is possibly linked to the choice of the ansatz, which moves the majority of variance to the edges.
          However, elimination of qubits causing distortion may in fact act contrary tothe very idea of QAE restoring the entire range of values in
          both latens and trash area. So this must be done with caution.
        - **Fix 09** The full series can now be reconstructured, with averaging of overlapping windows, and taking into consideration of window trimming.
        - **Fix 10** The systematic tests of the QAE has commensed and will continue.
    - V2.15 Test Summary, includes comments on Test 1, Test 2, ..., Test 6, and the Summary notebook
      - Observations:
        - The previous tests (such as v2.14_merged_ts), focused primarily on perfecting QAE where the trainnig and test sets were drawn from the same data sample.
          It seemed that the recovered input patterns had some edge distortion, i.e. increase in variance near qubit 0 and max, removal of which greatly improved the QAE accuracy.
          What was not obvious at the time was the fact that the distortion would most commonly occur in the trash area.
          This also means that by trimming the trash-related window components carried the majority of noise, and with a small trash area, the latent area was trained to approximate an identity.
        - In Test 1, the training sample and a test data were drawn from different sample types, however, both were of an oscillating nature, but of a different complexity and amplitude.
          The initial intuition indicated that the recovered data would match the patterns occuring in training data. However, all data recovered would match that of a test data sample -
          this is what QAE is in fact supposed to do.
        - Test 2a was further testing the hypothesis that QAE would recover the patterns present in a test sample. However, when the types of training and test samples are significantly different, i.e.
          sinusuidal and polynomila (almost linear differences betweel data points), the QAE had difficuties replicating the polynomial input. This could possibly be attributed to the QAE input coding
          which consisted of qubit rotations, and which favour Fourier style of data processing rather than that typical of regression or polynomial data analysis.
          The Test 2b seems to indicate that the QAE is not suitable for learning linear relationships, inspite of excellent cost achieved at the end - note that Test 2c excludes overtrainnig.
          It is also worth noting the errors generated by simply measuring the near-linear windows data, their R2 is as low (negative!) as that of the recovered signal!
        - Test 3 further confirmed the QAE ability to correctly recover test data, as long as the training data matched its type. However, having used the random oscillations in training
          data, it was a great surprise to see the QAE abillity to correctly recover test data. The hypothesis at this point of time was that QAE, without any intelligence, simply replicates i
          qubits spanning the latent area.
        - Test 4 again looked at polynomial data with near-linear test set. The results show very poor alignment (in terms of R2) between the orginal (but differenced) data and its recovered input.
          As before in Test 2, it showed that the error generated by QAE for input recovered on output is of the same magnitude as that generated by the circuit consisting of encoded input alone.
          The variance of measured input and the output values around the expected windows values look dramatic, however, this is illusory as the default y range in the chart is very small.
          It is worth noting that when looking at the results in the scale of the original data, their MAE and RMS are relatively small.
        - Tests 5, 6 and 7 aimed to verify whether or not the QAE was able to abstract the patters learnt from the training data and transfer this knowledge to test data.
          Data used for these experiments involved the USA beer sales over several weeks. In the tests, the trash area was progressively enlarged from 2 qubits (leaving 6 qubits in a latent area),
          to 3 (with 5 latent qubits) and then 4 (with 4 latent qubits). It was evident that the QAE was able to approximately recover the input, however, the errors would enlarge (as evidenced by
          R2, RMSE, MAE and MAPE measurement).
        - Test 8 was conducted to ensure that the latent space did not simply produce a copy of the input, which was found not to be the case.
        - <font color="green">Test 9 was to replace swap test with in-line test and a new cost function cost=1-P(|0>^t) where t is the size of trash. The approach resulted in faster execution but lower accuracy.</font>
        - In summary, the QAE works well. It is able to abstract training data, to allow recovery of previously unseen input data on its output. It is the richness of training data set, which determines
          the accuracy of QAE processing with test data, rather than the similarity of the two data sets. However, a completely different types of samples used in training and testing (such as
          oscillating vs linear) will prevent QAE to perform with any degree of accuracy.
    - V2.16 Development of the full-QAE which could then be used to support noise elimination, and allow the full input vs output comparison
      - Issues noted:
          - **Issue 17** Training of the full-QAE is ***50 times slower*** than of the half-QAE.
            The reasons for this is probably due to the Swap Test on 8 qubits rather than on just the trash space.
            Another likely reason is that the Swap Test is comparing arbitrary states rather than |0> states.
            Note that a Swap Test may not be effective when the states are close to |+>, which is our encoding of zero
            (see Siegelwax, B.N., 2020. The Simplest Way to Compare Single-Qubit Quantum States. Medium).
            A more efficient swap test may need to be used. Two possible solutions are considered at this stage:<br/>
            (1) train the model in two stages (first as half-QAE for noisy data, then as full-QAE for noisy vs clean data);<br/>
            (2) use a destructive swap on one qubit for state swapping and one ancilla for collection and measurement;<br/>
            (3) somehow reduce comparison of arbitrary states to zero states.e.g. by rotating all qubits back and then checking for |0>.
    - V2.17 Full-QAE with de-noising
      - Tests completed:
          - Value scaling was tested, large interval generates incorrect TS
          - Using full-QAE as a simple encoder S(x) A(x) S(x) with Swap Test to zero works, however, results are not as good as with half-QAE.
          - Three tests conducted to test the full-QAE performance with varying sizes of latent space, i.e. 2.17a (6+2), 2.17b (5+3), 2.17c (4+4)
      - Issues noted:
          - Issue 18 (Completed) Noise embedded in a circuit is not edequate as it is fixed for the entire duration of training.
            Noise needs to vary for all instances of training data instead.
          - Issue 19 (Completed) Scaling of pure and noisy signals seems to be incorrect, noise seems to have reduced amplitude of the signal.
            At the moment noise is added to the original signal. However, after differencing it seems to be distorted - this will affect the work of QAE in denoising.
            Perhaps noise needs to be added after differencing, so that it is not subtracted or amplified?
          - Issue 20 (Completed) At the moment the full-QAE incorrectly implements a decoder as an encoder-dag, it should be a separate block with its own parameters,
            which unfortunately will double execution time and slow it even more.
          - Issue 21 (Completed) Full-QAE does not learn to produce clean data from noisy data, it seems to pass noise from input to output.
            This may be related to the fact that optimisation error is higher than the discrepancy between noisy and pure data.
          - Issue 22 (Completed) Replace the expensive QAE cost function, with a simple P(|0>^n), which can be calculated as the count of all zeros / number of shots.
            Idea - aim is to maximise zeros, so measure all qubits, in cost function collect result of forward, check result[0] (all zeros), max this!
      - Fixes completed:
          - **Fix 03, 18** Now the noise is injected into the TS itself for training or testing
          - **Fix 19** Scaling to interval (-1, 1) results in incorrect interpretation of measurements (due to encoding), should not be attempted.
            Scaling with (-0.5, +0.5) works better.
    - V2.18 Full-QAE with de-noising and new cost function cost=1-P(|0>^q) (q=the number of qubits)
      - Observations:
          - The new full-QAE is twice as fast
          - It takes half the number of qubits
          - It is marginally less accurate
          - Does not address issue 21 (and possibly 20)
      - Fixes completed:
          - **Fix 22** The new cost function cost=1-P(|0>^q) (q=the number of qubits) has been implemented and tested.
    - V2.19 Full-QAE has a separate encoder and decoder
      - Observations:
          - The new full-QAE with separate encoder/decoder performs slightly better than encoder/encoder-dg
          - It is still unable to reproduce good TS from bad ones (issue 21), perhaps it needs on input a mix of good and noisy data.
      - Fixes completed:
          - **Fix 20** Full-QAE encoder is followed by an independent decoder-dg, same structure as encoder but in reverse order with different parameters.
    - V2.20 Implemented a new cost function which counts the number of 1s in measurements, aiming to eliminate them
      - Observations:
          - The optimisation very rapidly converges to lower values
          - However, it takes lots of iterations to achieve minimum
          - However, the minimum does not seem to be what is needed
          - The reconstructed curve is not changing much
    - V2.21 Going back to cost function 1-P(|0>^n), drastic reduction in data samples used for QAE training and validation
      - Observations:
          - Data sample was reduced to 29 examples for training and 10 for validation
          - The model seems to train quicker and smoother but requires more iterations (500)
          - The model actually generalises away from its training data and learns pure data from noise
          - Performance metrics are reasonable but not great
          - Much better results than before
      - Fixes completed:
          - **Fix 21** The model learns to reconstruct pure data windows from noisy data
      - Future work:
          - Test the model performance on other (simpler) data sets
          - Test the model performance with the cost function minimising the 1s in measurements
          - Try pretraining the encoder with half-QAE and stage decoder training (this may improve the training speed and address the only outstanding **Issue 17**)

# Software in use

In [ ]:
!pip list | grep -E 'qiskit|torch'

In [ ]:
import qiskit.tools.jupyter

# Note that Qiskit Terra and ML were compiled in from sources
%qiskit_version_table

# As this code was derived from Qiskit tutorial retain the following
%qiskit_copyright